In [1]:
import numpy as np  # linear algebra
import cv2 # opencv
import matplotlib.pyplot as plt # image plotting
# keras
from keras import Sequential # model 
from keras.layers import Flatten, Dense # nueral layers
from keras.applications.vgg16 import VGG16  # image classifying architecture
from keras.preprocessing.image import ImageDataGenerator  # data generator from images

In [2]:
train_dir = "E:/Nueral_Model/mask/Face Mask Dataset/Train" # directory for training images 
test_dir = "E:/Nueral_Model/mask/Face Mask Dataset/Test"  # directory for test images
val_dir = "E:/Nueral_Model/mask/Face Mask Dataset/Validation"  # directory for validation images

In [3]:
# Data preprocessing

datagenerator = ImageDataGenerator(
    rescale=1.0 / 255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2  # recscaling the image to 1/255 , with zoom range and horizontal flip 
)
# Train data
train_generator = datagenerator.flow_from_directory(  # making train generator to flow data from directory in a target size of 128,128 and in a batch of 32
    directory=train_dir, target_size=(128, 128), class_mode="categorical", batch_size=32
)

# Validation data
val_generator = datagenerator.flow_from_directory(
    directory=val_dir, target_size=(128, 128), class_mode="categorical", batch_size=32
)
# Test data
test_generator = datagenerator.flow_from_directory(
    directory=val_dir, target_size=(128, 128), class_mode="categorical", batch_size=32
)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [4]:
# Initializing the VGG16 model
# using the VGG 16 with the nest weights possible "imagenet"  in a shape of 128 ,128 
vgg16_model = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3)) 
for layer in vgg16_model.layers:
    layer.trainable = False  # making the layers in the VGG 16 untrainable as it will retrain the model


In [5]:

model1 = Sequential()  # initializing a sequential model
model1.add(vgg16_model)  # adding vgg16 model architecture to our model 
model1.add(Flatten())  # flattening the layer
model1.add(Dense(2, activation="sigmoid"))  # adding dense layer with activation function as "sigmoid"
model1.summary() # getting the summary of the model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 2)                 16386     
                                                                 
Total params: 14,731,074
Trainable params: 16,386
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# Compiling the model with the best optimizer for the problem 
model1.compile(optimizer="adam", loss="categorical_crossentropy", metrics="accuracy")

In [7]:
# fitting or training the model and storing its training history inside model1_history 
model1_history = model1.fit(
    train_generator, # training data 
    steps_per_epoch=len(train_generator) // 32,  # steps per epoch 
    epochs=20,                                    # total number of epoch 
    validation_data=val_generator,               # validation data
    validation_steps=len(val_generator) // 32,   # validation steps
)

Epoch 1/20
9/9 [==============================] - 159s 17s/step - loss: 0.4363 - accuracy: 0.8021
Epoch 2/20
9/9 [==============================] - 155s 17s/step - loss: 0.1656 - accuracy: 0.9549
Epoch 3/20
9/9 [==============================] - 153s 17s/step - loss: 0.1202 - accuracy: 0.9583
Epoch 4/20
9/9 [==============================] - 149s 17s/step - loss: 0.0902 - accuracy: 0.9688
Epoch 5/20
9/9 [==============================] - 148s 16s/step - loss: 0.0688 - accuracy: 0.9722
Epoch 6/20
9/9 [==============================] - 149s 16s/step - loss: 0.0895 - accuracy: 0.9618
Epoch 7/20
9/9 [==============================] - 149s 16s/step - loss: 0.0485 - accuracy: 0.9826
Epoch 8/20
9/9 [==============================] - 169s 19s/step - loss: 0.0433 - accuracy: 0.9896
Epoch 9/20
9/9 [==============================] - 151s 16s/step - loss: 0.0536 - accuracy: 0.9792
Epoch 10/20
9/9 [==============================] - 147s 16s/step - loss: 0.0359 - accuracy: 0.9931
Epoch 11/20
9/9 [==

In [9]:
# Evaluate model performance on test data
model_loss, model_acc = model1.evaluate(test_generator)
print("Model has a loss of %.2f and accuracy %.2f%%" % (model_loss, model_acc*100))

25/25 [==============================] - 430s 16s/step - loss: 0.0377 - accuracy: 0.9900
Model has a loss of 0.04 and accuracy 99.00%


In [8]:
model1.save('./miniprojectmask.h5')